<a href="https://colab.research.google.com/github/10zinchosang/Summer2022-DataAnalytics/blob/main/Final_Project_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Market Data Analysis

# Task 1: Identify the research question

### How have stocks from various industries fared during the covid pandemic?

I am asking this question because there were many winners and losers in terms of large scale companies during the pandemic. I will analyze how various big players did in the market.

## Overview/plan

I will analyze historical stock data from Yahoo Finance for handpicked companies. I will then clean and wrangle the data for interesting features. Then I will make visualizations and a linear regression model at the end.

# Task 2: Load the Libraries

In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install yfinance
import yfinance as yf
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


I chose the standard libraries plus yfinance so I can directly choose which stocks I want and plotly for more graphing options.

# Task 3: Load the Data

First I chose the stocks I want to look at. For the sake of diversity I chose: Pfizer(PFE), Target(TGT), Netflix(NFLX), Clorox(CLX), and ExxonMobil(XOM). Then I chose the time frame of January 1, 2019 to July 31, 2022 so I can have a buffer for before and "after" the pandemic, if we can even say it's over. Lastly, I created global variable dataframes named after the stock tickers using a for loop.

In [84]:
stock_list = ['PFE','TGT','NFLX','CLX','XOM']

start = datetime(2020,1,1)
end = datetime(2022,7,31)

for stock in stock_list:
    globals()[stock] = yf.download(stock,start,end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


I used this resource for helping me extract historical data using the yfinance library: https://www.learnpythonwithrune.org/read-historical-prices-from-yahoo-finance-with-python/

# Task 4: EDA (Exploratory Data Analysis)

## Basic EDA

In [85]:
PFE.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,37.286530,37.333965,36.888046,37.134724,33.538967,16514072
2020-01-03,36.736244,37.229603,36.688805,36.935486,33.359020,14922848
2020-01-06,36.831120,37.001900,36.717270,36.888046,33.316170,15771951
2020-01-07,37.115749,37.125237,36.698292,36.764706,33.204784,20108107
2020-01-08,36.774193,37.210625,36.764706,37.058823,33.470421,16403507


In [86]:
PFE.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-07-25,51.340000,51.880001,51.340000,51.770000,51.371384,12995800
2022-07-26,51.840000,52.849998,51.709999,52.299999,51.897305,16632500
2022-07-27,51.939999,52.250000,51.189999,51.950001,51.549999,19203500
2022-07-28,50.849998,52.330002,49.099998,50.720001,50.720001,38984900
2022-07-29,50.560001,50.709999,49.279999,50.509998,50.509998,25035400


In [87]:
PFE.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 649 entries, 2020-01-02 to 2022-07-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       649 non-null    float64
 1   High       649 non-null    float64
 2   Low        649 non-null    float64
 3   Close      649 non-null    float64
 4   Adj Close  649 non-null    float64
 5   Volume     649 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 35.5 KB


In [88]:
PFE.describe()

,Open,High,Low,Close,Adj Close,Volume
count,649.000000,649.000000,649.000000,649.000000,649.000000,6.490000e+02
mean,41.413061,41.900769,40.913590,41.414952,39.598182,3.215806e+07
std,7.622492,7.792721,7.494620,7.669150,8.347395,1.779226e+07
min,27.286528,28.064516,26.451612,27.030361,24.662390,1.084640e+07
25%,35.455406,35.790001,35.066414,35.417458,32.925297,2.173300e+07
50%,38.880001,39.110001,38.520000,38.790001,36.994831,2.749053e+07
75%,48.500000,49.279999,47.770000,48.750000,47.875065,3.688980e+07
max,60.599998,61.709999,59.830002,61.250000,59.831848,2.301539e+08


# Task 5: Data wrangling

I will now create a large dataframe that has all the closing prices for the 5 stocks. This will allow me to create a large graph including all 5 closing stock prices. We must merge the dataframes one by one on "Date" which is their index. But before I do that I will rename all the "Close" columns of each dataframe so they include the stock tickers as well.



## Renaming "Close" columns

In [89]:
for stock in stock_list:
    globals()[stock].rename(columns = {'Close': str(stock)+' Close'}, inplace = True)

In [90]:
PFE.columns

Index(['Open', 'High', 'Low', 'PFE Close', 'Adj Close', 'Volume'], dtype='object')

In [91]:
TGT.columns

Index(['Open', 'High', 'Low', 'TGT Close', 'Adj Close', 'Volume'], dtype='object')

## Create copies of dataframes

In [92]:
PFE_copy = PFE.copy()
TGT_copy = TGT.copy()
NFLX_copy = NFLX.copy()
CLX_copy = CLX.copy()
XOM_copy = XOM.copy()

## Drop the columns

In [93]:
copy_list = []

for stock in stock_list:
    copy = str(stock)+"_copy"
    copy_list.append(copy)

In [94]:
copy_list

['PFE_copy', 'TGT_copy', 'NFLX_copy', 'CLX_copy', 'XOM_copy']

In [95]:
for stock in copy_list:
    globals()[stock].drop(columns=['Open','High','Low','Adj Close', 'Volume'], inplace=True)

## Reset index for merging

In [96]:
for stock in copy_list:
    globals()[stock].reset_index(inplace=True)

In [97]:
PFE_copy.head()

,Date,PFE Close
0,2020-01-02,37.134724
1,2020-01-03,36.935486
2,2020-01-06,36.888046
3,2020-01-07,36.764706
4,2020-01-08,37.058823


## Merge the Close columns

In [98]:
merged1 = pd.merge(PFE_copy,TGT_copy)

In [99]:
merged2 = pd.merge(merged1,NFLX_copy)

In [100]:
merged3 = pd.merge(merged2,CLX_copy)

In [101]:
merged = pd.merge(merged3,XOM_copy)

In [102]:
merged.head()

,Date,PFE Close,TGT Close,NFLX Close,CLX Close,XOM Close
0,2020-01-02,37.134724,126.070000,329.809998,152.610001,70.900002
1,2020-01-03,36.935486,124.760002,325.899994,152.910004,70.330002
2,2020-01-06,36.888046,123.580002,335.829987,153.369995,70.870003
3,2020-01-07,36.764706,123.800003,330.750000,151.520004,70.290001
4,2020-01-08,37.058823,123.400002,339.260010,151.970001,69.230003


# Task 5: Data Visualization

I created a little "function" that allows the user to pick what stock they would like to graph the closing price and volume for.

I used this site for guidance on how to plot using plotly: https://pythoninoffice.com/draw-stock-chart-with-python/

In [103]:
ticker = str(input("Please enter a stock ticker out of ['PFE','TGT','NFLX','CLX','XOM']: "))
close = ticker + " Close"

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=globals()[ticker].index,y=globals()[ticker][close],name='Closing Price'),secondary_y=False)
fig.add_trace(go.Bar(x=globals()[ticker].index,y=globals()[ticker]['Volume'],name='Volume'),secondary_y=True)
fig.update_layout(title_text='Closing Price and Volume of ' + ticker + ' from (1/1/19 - 7/31/22)')
upper = max(globals()[ticker]['Volume']) * 3
fig.update_yaxes(range=[0,upper],secondary_y=True)
fig.show()

Please enter a stock ticker out of ['PFE','TGT','NFLX','CLX','XOM']: TGT


Looking through the stocks, we can make the following observations:
1. Pfizer experienced growth overall during the pandemic, but had many dips and peaks.
2. Target experienced large growth throughout the pandemic, but dropped early 2022.
3. Netflix experienced large growth througout the pandemic, but dropped a lot beginning of 2022.
4. Clorox experienced large growth throughout the pandemic, but has returned to prepandemic price.
5. ExxonMobil experienced loss throughout the pandemic, but has now surpassed prepandemic price.

# Task 6: Train Test Split on Pfizer

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [105]:
X = PFE[['Open','Volume']]
y = PFE['PFE Close']

In [106]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

# Task 7: Linear Regression on PFE

In [107]:
regressor = LinearRegression()

In [108]:
regressor.fit(X_train,y_train)

LinearRegression()

# Task 8: Evaluate the model

In [109]:
regressor.score(X, y)

0.9927425935689755

# Conclusion

In [110]:
fig2 = make_subplots()
fig2.add_trace(go.Scatter(x=PFE.index,y=PFE['PFE Close'],name='Pfizer'))
fig2.add_trace(go.Scatter(x=TGT.index,y=TGT['TGT Close'],name='Target'))
fig2.add_trace(go.Scatter(x=NFLX.index,y=NFLX['NFLX Close'],name='Netflix'))
fig2.add_trace(go.Scatter(x=CLX.index,y=CLX['CLX Close'],name='Clorox'))
fig2.add_trace(go.Scatter(x=XOM.index,y=XOM['XOM Close'],name='ExxonMobil'))
fig2.update_layout(title_text='Closing Price of 5 stocks from (1/1/19 - 7/31/22)')
fig2.show()

There were many big winners and losers in terms of stocks during the pandemic. Many of them both winners and losers. Overall we saw in increase in all of these stocks because of various niches they fill: vaccines(Pfizer), affordable retail(Target), at home entertainment(Netflix), cleaning supplies(Clorox), and gas(ExxonMobil).

ExxonMobil is the only stock that failed during the pandemic, but has surpassed prepandemic price out of the 5 stocks. This can mainly be attributed to the geo-political situation we are facing involving the Russia and Ukraine war. Supply chain failures negatively affected all stocks, but the Russia oil sanctions have pushed investors to invest in energy such as ExxonMobil.

If I could add more to this project, I would look at what industries/stocks were affected by the Russia and Ukraine war situation.